## A minimalist example for recovering sparse graphs using `uGLAD`

Fitting uGLAD on a erdos-renyi random sparse graph with samples obtained from a corresponding multivariate Gaussian distribution.    

### About `uGLAD` 
An unsupervised learning based approach to recover sparse graphs. This work proposes `uGLAD` which is a unsupervised version of a previous `GLAD` model (GLAD: Learning Sparse Graph Recovery (ICLR 2020 - [link](<https://openreview.net/forum?id=BkxpMTEtPB>)).  

Key benefits & features:  
- Solution to Graphical Lasso: A better alternative to solve the Graphical Lasso problem as
    - The neural networks of the uGLAD enable adaptive choices of the hyperparameters which leads to better performance than the existing algorithms  
     - No need to pre-specify the sparsity related regularization hyperparameters    
    - Requires less number of iterations to converge due to neural network based acceleration of the unrolled optimization algorithm (Alternating Minimization)    
    - GPU based acceleration can be leveraged  
- Glasso loss function: The loss is the logdet objective of the graphical lasso `1/M(-1*log|theta|+ <S, theta>)`, where `M=num_samples, S=input covariance matrix, theta=predicted precision matrix`.  
- Ease of usability: Matches the I/O signature of `sklearn GraphicalLassoCV`, so easy to plug-in to the existing code.  

In [1]:
import os, sys
# reloads modules automatically before entering the 
# execution of code typed at the IPython prompt.
%load_ext autoreload
%autoreload 2
# install jupyter-notebook in the env if the prefix does not 
# show the desired virtual env. 
print(sys.prefix)
import warnings
warnings.filterwarnings('ignore')

/home/harshx/anaconda3/envs/uGLAD


In [2]:
import torch
torch.__version__

'1.10.1'

### Create sample data

In [17]:
from uGLAD.utils.prepare_data import get_data
from uGLAD.utils.metrics import reportMetrics

# Xb = samples batch, trueTheta = corresponding true precision matrices
Xb, true_theta = get_data(
    num_nodes=10, 
    sparsity=0.2, 
    num_samples=500, 
    batch_size=1,
    eig_offset=1, 
    w_min=0.5,
    w_max=1
)
print(f'true_theta: {true_theta.shape}, Samples {Xb.shape}')

true_theta: (1, 10, 10), Samples (1, 500, 10)


### The uGLAD model

Learning details:  
1. Initialize learnable `GLAD` parameters  
2. Run the GLAD model  
3. Get the glasso-loss  
4. Backprop  

Possible solutions if `uGLAD` does not converge:  
1. Increase number of training EPOCHS
2. Lower the learning rate    
3. Please re-run. This will run the optimization with different initializations  
4. Change the INIT_DIAG=0/1 in the `GLAD` model parameters  
5. Increase `L`, the number of unrolled iterations of `GLAD`

### Running the uGLAD-Direct mode

- Directly optimize the uGLAD model on the complete data X
- Optimizes the model to minimize the glasso-loss on X 

In [21]:
from uGLAD import main as uG

# Initialize the model
model_uGLAD = uG.uGLAD_GL()  

# Fit to the data
model_uGLAD.fit(
    Xb[0],
    centered=False,
    epochs=250,
    lr=0.002,
    INIT_DIAG=0,
    L=15,
    verbose=False, 
    k_fold=0  # Direct mode
)  

# Comparing with true precision matrix
compare_theta_uGLAD = reportMetrics(
        true_theta[0], 
        model_uGLAD.precision_
    )
print(f'uGLAD: {compare_theta_uGLAD}')

Running uGLAD
Direct Mode
Total runtime: 9.464847803115845 secs

uGLAD: {'FDR': 0.0, 'TPR': 1.0, 'FPR': 0.0, 'SHD': 0, 'nnzTrue': 13, 'nnzPred': 13, 'precision': 1.0, 'recall': 1.0, 'Fbeta': 1.0, 'aupr': 0.9999999999999998, 'auc': 1.0}


### Running the uGLAD-CV mode 

- Finds the best model by doing cross-fold validation on the input samples X
- Chooses the model which performs best in terms of glasso-loss on held-out data
- More conservative than the direct mode

In [26]:
from uGLAD import main as uG

# Initialize the model
model_uGLAD = uG.uGLAD_GL()  

# Fit to the data
model_uGLAD.fit(
    Xb[0],
    centered=False,
    epochs=250,
    lr=0.002,
    INIT_DIAG=0,
    L=15,
    verbose=False,
    k_fold=3
)  

# Comparing with true precision matrix
compare_theta_uGLAD = reportMetrics(
        true_theta[0], 
        model_uGLAD.precision_
    )
print(f'uGLAD: {compare_theta_uGLAD}')

Running uGLAD
CV mode: 3-fold
Total runtime: 36.47530436515808 secs

uGLAD: {'FDR': 0.13333333333333333, 'TPR': 1.0, 'FPR': 0.0625, 'SHD': 2, 'nnzTrue': 13, 'nnzPred': 15, 'precision': 0.8666666666666667, 'recall': 1.0, 'Fbeta': 0.9285714285714286, 'aupr': 0.9999999999999998, 'auc': 1.0}


### Comparison with sklearn's GraphicalLassoCV

In [24]:
from sklearn.covariance import GraphicalLassoCV

model_BCD = GraphicalLassoCV().fit(Xb[0])
# Compare with theta
compare_theta_BCD = reportMetrics(
    true_theta[0], 
    model_BCD.precision_
)
print(f'BCD: {compare_theta_BCD}')

BCD: {'FDR': 0.5806451612903226, 'TPR': 1.0, 'FPR': 0.5625, 'SHD': 18, 'nnzTrue': 13, 'nnzPred': 31, 'precision': 0.41935483870967744, 'recall': 1.0, 'Fbeta': 0.5909090909090909, 'aupr': 0.9999999999999998, 'auc': 1.0}
